In [1]:
import pandas as pd

In [3]:
file = "data/access-export/tblTokens_ht-26603.xlsx"

In [4]:
df = pd.read_excel(file)

In [5]:
df

,ID,spelling,leftcontext,plainword,PoS,label,comments,rightcontext,textsource,cluster,file,medialsuffix,finalsuffix,spelling2,rightonset
0,&d-1,&de,arise&d_VBP ._. for&get_VBI al_Q &tis_D folc_N...,deorewr&de,ADJ,DS?,vowel missing,dohter_N sei&d_VBP daui&d_NPR &te_D witege_N ....,cmhali.m1,rθ,ecce_ALLth_IN_Okt27,NaN,NaN,CCE,NaN
1,&d-10,&de,in_P his_PRO$ seruise_N &_CONJ habbe_HV se_ADV...,murh&de,N,L,NaN,&trof_ADV&P &_CONJ licunge_N i_P &tin_PRO$ heo...,cmhali.m1,rθ,ecce_ALLth_IN_Okt27,NaN,NaN,CCE,θ
2,&d-1000,&d,leas_ADJ wind_N &tt_C we_PRO i_P &tis_D worlt_N,liuie&d,VBP,C?,NaN,._. leaue&d_VBI &tt_C leas_ADJ is_BEP ._. &_CONJ,cmjulia.m1,NaN,ecce_ALLth_IN_Okt27,NaN,NaN,CEC,#
3,&d-10000,&de,neauer_ADV in&geong_N ._. ach_CONJ tap_VBI him...,$&de,D,C,NaN,sculle_N ._. for_CONJ he_PRO is_BEP $arch_ADJ ...,NaN,NaN,ecce_ALLth_IN_Okt27,NaN,NaN,NaN,NaN
4,&d-10001,&de,hit_PRO waxe&d_VBP ._. Vnder_VBI21 stont_VBI22...,$&de,PRO,C,NaN,seolf_N ._. asich&de_D&N &tt_C &tu_PRO sist_VB...,NaN,NaN,ecce_ALLth_IN_Okt27,NaN,NaN,NaN,NaN
5,&d-10002,&de,Vnder_VBI21 stont_VBI22 &tis_D $bi_P $&de_PRO ...,asich&de,D&N,D?,vocalised?,&tt_C &tu_PRO sist_VBP o&der_CONJ an_D lewi_AD...,cmancriw-1.m1,NaN,ecce_ALLth_IN_Okt27,NaN,NaN,CCE,θ
6,&d-10003,&de,walde_MD ._. Monicunnes_Q&N$ fondunge_N is_BEP...,feor&de,ADJ,D,NaN,"dale_N ._, Misliche_ADJ frouren_NS &_CONJ $Mon...",cmancriw-1.m1,rθ,ecce_ALLth_IN_Okt27,NaN,&de,CCE,NaN
7,&d-10004,&de,as_P ich_PRO &ter_ADV seide_VBD $Her_ADV bigin...,fif&de,ADJ,LD?,vowel change?,dale_N $of_P schrift_N Twa_NUM &tinges_NS neom...,cmancriw-1.m1,fθ,ecce_ALLth_IN_Okt27,NaN,NaN,CCE,NaN
8,&d-10006,&de,schrif_N schule_MD beon_BE &te_C beo_BEP of_P ...,streng&de,N,LD?,NaN,&_CONJ for_FOR to_TO schawen_VB hit_PRO bet_AD...,cmancriw-1.m1,ŋgθ,ecce_ALLth_IN_Okt27,NaN,NaN,CCCE,NaN
9,&d-10007,&de,him_PRO to_TO sungen_VB as_P &tach_P he_PRO he...,streng&de,N,LD?,NaN,"._, &te_C naue&d_NEG&HVP nan_Q Mid_P alle_Q bu...",cmancriw-1.m1,ŋgθ,ecce_ALLth_IN_Okt27,NaN,NaN,CCCE,θ


In [11]:
# create necessary concept-schemes
ecce_pos,_ = SkosConceptScheme.objects.get_or_create(dc_title="ecce-pos")

In [8]:
pos = set(df['PoS'])

In [13]:
for x in pos:
    temp_variable,_ = SkosConcept.objects.get_or_create(pref_label=x)
    temp_variable.save()
    temp_variable.scheme = [ecce_pos]

In [14]:
df_slice = df.ix[:100]

In [ ]:
# ceate objects
for i, row in df.iterrows():
    temp_object,_ = Token.objects.get_or_create(
        legacy_id=row['ID'],
        spelling=row['spelling'],
        left_context=row['leftcontext'],
        plain_word=row['plainword'],
        comments=row['comments'],
        right_context=row['rightcontext'],
        file=row['file'],
        medial_suffix=row['medialsuffix'],
        final_suffix=row['finalsuffix'],     
    )
    try:
        temp_pos = SkosConcept.objects.get(pref_label=row['PoS'])
        temp_object.pos=temp_pos
        temp_object.save()
    except:
        pass
    try:
        temp_fk = TokenLabel.objects.get(label=row['label'])
        temp_object.label=temp_fk
        temp_object.save()
    except:
        pass
    try:
        temp_fk = Text.objects.get(text=row['textsource'])
        temp_object.text_source=temp_fk
        temp_object.save()
    except:
        pass
    try:
        temp_fk = Cluster.objects.get(consonant=row['cluster'])
        temp_object.cluster=temp_fk
        temp_object.save()
    except:
        pass
    try:
        temp_fk = SchwaPresent.objects.get(spelling=row['spelling2'])
        temp_object.spelling2=temp_fk
        temp_object.save()
    except:
        pass
    try:
        temp_fk = OnSet.objects.get(rightonset=row['rightonset'])
        temp_object.rightonset=temp_fk
        temp_object.save()
    except:
        pass

In [17]:
#for x in Token.objects.all():
 #   x.delete()